In [3]:
import numpy as np
import pandas as pd
from typing import Union, Optional
from pathlib import Path

In [5]:
class OptimaOutput:
    def __init__(self, job_name: Union[str, Path],
                 optima_path: Optional[Union[str, Path]] = None):

        self.job_name = str(job_name)
        if optima_path == None:
            self.optima_path = Path("/home/szupernikusz/TDK/Opt/outputs")
        self.job_folder = self.optima_path / job_name
        file_path = self.job_folder / "mechanismInfo.txt"
        try:
            with open(file_path, "r") as f:
                self.mech_info = f.read()
        except Exception as e:
            raise e


class OptimaMechtest(OptimaOutput):
    def __init__(self, job_name: Union[str, Path],
                 optima_path: Optional[Union[str, Path]] = None,
                 errf_type: Union[str, list[str]] = "default"):     # Union means that it can be either this or that type
        super().__init__(job_name, optima_path)                     # i.e., now either str, or list[str]

        self._errf_files = {
           "default": "errfValues",
           "data_series": "errfValues_by_data_series", 
           "points": "errfValues_by_points",
           "species": "errfValues_by_species"
            }

        if (self.job_folder / "debug").exists():
            self.all_data = {}
            for csv_data in (self.job_folder / "debug").glob("*.csv"):
                try:
                    self.all_data[csv_data.stem] = pd.read_csv(csv_data)    # key - value pairs --> key is csv filename without
                                                                            # the extension, value is the .csv as a pd df
                except Exception as e:                                      # i.e., we read all .csv files into a dict (= all_data)
                    print(e)

        if (self.job_folder / "mechTestResults_BCRN6.csv").exists():
            all_sheets_dP = pd.read_csv(self.job_folder / "mechTestResults_BCRN6.csv",  # dP as in the info in dataPoints
                                        header=None,
                                        delimiter=';',
                                        index_col=False,
                                        names=['xml', 'time_point', 'species', 'dP_val', 'sim_val'])
            self.get_the34(all_sheets_dP=all_sheets_dP)

        try:
            if isinstance(errf_type, str):                                  # isinstance(alma, type1) checks if alma is of type 'type1'
                file_name = self._errf_files.get(errf_type, "errfValues")   # i.e., now, it checks if errf_type is a string or not
                self.errfValues = pd.read_csv(                              # since according to the type def. above, it could
                    self.job_folder / file_name,                            # either be a string or a list of strings
                    header = 9,
                    delimiter=r"\s+"                                        # reading the errfValues file as a .csv, omitting the first
                )                                                           # 9 lines of the file (as the header)
            else:
                self.errfValues = {}
                for e_type in errf_type:
                    file_name = self._errf_files.get(errf_type, "errfValues")   # this is a dict() method, such that
                    self.errfValues[e_type] = pd.read_csv(                      # .get(key, default_value) returns the key
                        self.job_folder / file_name,                            # if it's in the dict(), or the default_value
                        header = 9,                                             # if the key is not in the dict()
                        delimiter=r"\s+"
                    )

            if (self.job_folder / "sigmas").is_file():
                self.sigmas = pd.read_csv(
                    self.job_folder / "sigmas",
                    skiprows = [0,2],
                    delimiter=r"\s+"
                )
        except Exception as e:
            raise e

        stac_eq_df = pd.concat({k: v.iloc[-1] for k, v in self.all_data.items()}, axis=1)       # makes a dict() with the
        followed34 = pd.concat({k: v.iloc[-1] for k, v in self.xml_dP.items()}, axis=1)         # same keys, the values is
        self.df_basal = stac_eq_df.iloc[3:-1].T * 1e12                                         # the last element i.e.,
        self.df_followed34 = followed34.T * 1e12                                               # t = last time_point

    def __str__(self):
        if not self.df_basal.empty and not self.df_followed34.empty:
            return f"Mech object was successfully generated with fields\ndf_basal: {self.df_basal.shape}\ndf_followed34: {self.df_followed34.shape}\nsigmas: {self.sigmas.shape}"
        elif self.df_basal.empty and not self.df_followed34.empty:
            return "Unsuccessful, error with df_basal"
        elif not self.df_basal.empty and self.df_followed34.empty:
            return "Unsuccessful, error with df_followed34"
        else:
            return "Mech object was not successfully created"

    def get_the34(self, all_sheets_dP):
        self.xml_dP: dict[str, pd.DataFrame] = {}
        time_point = 0
        species = 'ilyen_species_tuti_nem_lesz'
        for idx, row in all_sheets_dP.iterrows():
            xml_name = row.xml
            sim_val = row.sim_val

            if row.species == species:
                time_point = time_point + 1
            else:
                species = row.species
                time_point = 1
            
            if xml_name not in self.xml_dP.keys():
                self.xml_dP[xml_name] = pd.DataFrame()  # Initialize inner dict
            self.xml_dP[xml_name].loc[time_point, species] = sim_val


In [6]:
float(0.10e+01)

1.0